<a href="https://colab.research.google.com/github/wkdalsgh192/Coursera_Capstone/blob/master/Week%202)Segmenting_and_Clustering_Newyork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [7]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [0]:
with open('newyork_data.json') as json_data:
  newyork_data = json.load(json_data)

In [9]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]['geometry']

{'coordinates': [-73.84720052054902, 40.89470517661], 'type': 'Point'}

In [15]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns = column_names)

for data in neighborhoods_data:
  borough = data['properties']['borough']
  neighborhood = data['properties']['name']
  latitude = data['geometry']['coordinates'][1]
  longitude = data['geometry']['coordinates'][0]

  neighborhoods = neighborhoods.append({'Borough': borough,
                                       'Neighborhood': neighborhood,
                                       'Latitude': latitude,
                                       'Longitude': longitude}, ignore_index = True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [0]:
print("The dataframe has {} boroughs and {} neighborhoods".format(len(neighborhoods['Borough'].unique()), neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods


In [0]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'Ryu_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print("The geograpical coordinates of New York City are {}, {}".format(latitude, longitude))

The geograpical coordinates of New York City are 40.7127281, -74.0060152


In [0]:
NY_map = folium.Map(location = [latitude, longitude], zoom_start = 10)

for lat, lon, borough, neighbor in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
  label = '{},{}'.format(neighbor, borough)
  label = folium.Popup(label, parse_html = True)
  folium.CircleMarker([lat,lon], radius = 3, popup = label, color = 'green', fill = True, fill_color = '#32a852', fill_opacity = 0.6, parse_html = False).add_to(NY_map)

NY_map

In [18]:
manhtan = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop = True)
print(manhtan.shape[0])
print(manhtan.head())

40
     Borough        Neighborhood   Latitude  Longitude
0  Manhattan         Marble Hill  40.876551 -73.910660
1  Manhattan           Chinatown  40.715618 -73.994279
2  Manhattan  Washington Heights  40.851903 -73.936900
3  Manhattan              Inwood  40.867684 -73.921210
4  Manhattan    Hamilton Heights  40.823604 -73.949688


In [23]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent = 'Ryu_explorer')
location = geolocator.geocode(address)
lati = location.latitude
longi = location.longitude

Manhattan_map = folium.Map(location = [lati, longi], zoom_start = 12)

for la, lo, bo, ne in zip(manhtan['Latitude'],manhtan['Longitude'],manhtan['Borough'],manhtan['Neighborhood']):
  label = '{},{}'.format(bo, ne)
  label = folium.Popup(label, parse_html = True)
  folium.CircleMarker([la, lo], radius = 5, popup = label, color = 'green', fill = True, fill_color = '#32a852', fill_opacity = 0.6, parse_html = False).add_to(Manhattan_map)


Manhattan_map


In [0]:
manhtan.to_csv("/content/gdrive/My Drive/Coding/manhattan.csv", index = False)
manhtan = pd.read_csv("/content/gdrive/My Drive/Coding/manhattan.csv")
print("The file uploaded")

neighborhood_latitude =  manhtan.loc[0, 'Latitude']
neighborhood_longitude =  manhtan.loc[0, 'Longitude']
neighborhood_latitude

The file uploaded


40.87655077879964

In [0]:
CLIENT_ID = '3TJMKB3HVSZ30DJ5RHBBHSTFISRXCHVXZVU5R15RCP4UOS31' # my Foursquare ID
CLIENT_SECRET = 'IWGDMBCNVGZRCTFMIBVSWV0CM1VJYZYYTT44WP24X1RNZLYK' # my Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('my credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

radius = 500
LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

my credentails:
CLIENT_ID: 3TJMKB3HVSZ30DJ5RHBBHSTFISRXCHVXZVU5R15RCP4UOS31
CLIENT_SECRET:IWGDMBCNVGZRCTFMIBVSWV0CM1VJYZYYTT44WP24X1RNZLYK


'https://api.foursquare.com/v2/venues/explore?&client_id=3TJMKB3HVSZ30DJ5RHBBHSTFISRXCHVXZVU5R15RCP4UOS31&client_secret=IWGDMBCNVGZRCTFMIBVSWV0CM1VJYZYYTT44WP24X1RNZLYK&v=20180604&ll=40.87655077879964,-73.91065965862981&radius=500&limit=100'

In [0]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

results = requests.get(url).json()
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis = 1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Arturo's,Pizza Place,40.874412,-73.910271
1,Bikram Yoga,Yoga Studio,40.876844,-73.906204
2,Tibbett Diner,Diner,40.880404,-73.908937
3,Starbucks,Coffee Shop,40.877531,-73.905582
4,Dunkin',Donut Shop,40.877136,-73.906666


In [0]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [0]:
manhattan_venues = getNearbyVenues(names=manhtan['Neighborhood'],
                                   latitudes=manhtan['Latitude'],
                                   longitudes=manhtan['Longitude']
                                  )


Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [0]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3331, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop


In [0]:
neighborhoods = manhattan_venues.groupby('Neighborhood').count()
hoods_encoding = pd.get_dummies(manhattan_venues['Venue Category'], prefix = "", prefix_sep = "")
hoods_encoding['Neighborhood'] = manhattan_venues['Neighborhood']

columns = [hoods_encoding.columns[-1]] + list( hoods_encoding.columns[:-1])
hoods_encoding = hoods_encoding.loc[:, columns]
hoods_encoding.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Candy Store,Caribbean Restaurant,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Bookstore,College Cafeteria,College Theater,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shanghai Restaurant,Shipping Stor

In [0]:
hoods = hoods_encoding.groupby('Neighborhood').mean().reset_index()
hoods.head()

hoods.to_csv("/content/gdrive/My Drive/Coding/hoods.csv", index = False)

In [30]:
hoods = pd.read_csv("/content/gdrive/My Drive/Coding/hoods.csv")

for hood in hoods['Neighborhood']:
  print("-----"+hood+"-----")
  frequency = hoods[hoods['Neighborhood'] == hood].T.reset_index()
  frequency.columns = ['Name', 'Frequency']
  frequency = frequency.iloc[1:]
  frequency['Frequency'] = frequency['Frequency'].astype(float)
  frequency['Frequency'] = round(frequency['Frequency'], 2)
  print(frequency.sort_values('Frequency', ascending = False).reset_index(drop = True).head(5))
  print('\n')

-----Battery Park City-----
            Name  Frequency
0           Park       0.09
1    Coffee Shop       0.07
2          Hotel       0.05
3            Gym       0.04
4  Memorial Site       0.04


-----Carnegie Hill-----
             Name  Frequency
0     Coffee Shop       0.06
1     Pizza Place       0.06
2            Café       0.04
3  Cosmetics Shop       0.04
4     Yoga Studio       0.03


-----Central Harlem-----
                   Name  Frequency
0    Chinese Restaurant       0.06
1    African Restaurant       0.06
2  Gym / Fitness Center       0.04
3            Public Art       0.04
4   American Restaurant       0.04


-----Chelsea-----
                 Name  Frequency
0         Coffee Shop       0.06
1      Ice Cream Shop       0.05
2              Bakery       0.05
3  Italian Restaurant       0.05
4           Nightclub       0.04


-----Chinatown-----
                    Name  Frequency
0     Chinese Restaurant       0.08
1           Cocktail Bar       0.05
2                 B

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


num_top_venues = 10
indicators = ['st', 'rd', 'nd']

columns = ['Neighborhood']
for ind in np.arange(10):
  try:
    columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
  except:
    columns.append('{}th Most Common Venue'.format(ind+1))

venue_sorted = pd.DataFrame(columns = columns)
venue_sorted['Neighborhood'] = hoods['Neighborhood']

for ind in np.arange(venue_sorted.shape[0]):
  venue_sorted.iloc[ind, 1:] = return_most_common_venues(hoods.iloc[ind, :], num_top_venues)

venue_sorted.head()



,Neighborhood,1st Most Common Venue,2rd Most Common Venue,3nd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Coffee Shop,Hotel,Gym,Memorial Site,Italian Restaurant,Wine Shop,Shopping Mall,Sandwich Place,Food Court
1,Carnegie Hill,Coffee Shop,Pizza Place,Cosmetics Shop,Café,Japanese Restaurant,Grocery Store,French Restaurant,Yoga Studio,Bookstore,Bar
2,Central Harlem,African Restaurant,Chinese Restaurant,Bar,American Restaurant,French Restaurant,Seafood Restaurant,Gym / Fitness Center,Public Art,Boutique,Beer Bar
3,Chelsea,Coffee Shop,Italian Restaurant,Bakery,Ice Cream Shop,Nightclub,Hotel,Theater,American Restaurant,Seafood Restaurant,Cycle Studio
4,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bakery,Bubble Tea Shop,Spa,Optical Shop,Salon / Barbershop,Ice Cream Shop


In [45]:
kclusters = 3
hoods_clustering = hoods.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters = kclusters, init = 'k-means++', n_init = 10, random_state = 0).fit(hoods_clustering)
kmeans.labels_[0:10]


array([1, 1, 1, 2, 1, 2, 2, 1, 1, 2], dtype=int32)

In [53]:
venue_sorted.insert(0, 'Labels', kmeans.labels_)
df_hoods = venue_sorted
Final_hoods = manhtan.join(df_hoods.set_index('Neighborhood'), on='Neighborhood')
Final_hoods.head()

,Borough,Neighborhood,Latitude,Longitude,Labels,1st Most Common Venue,2rd Most Common Venue,3nd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Sandwich Place,Coffee Shop,Yoga Studio,Discount Store,Miscellaneous Shop,Steakhouse,Supplement Shop,Seafood Restaurant,Tennis Stadium,Gym
1,Manhattan,Chinatown,40.715618,-73.994279,1,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bakery,Bubble Tea Shop,Spa,Optical Shop,Salon / Barbershop,Ice Cream Shop
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Deli / Bodega,Mobile Phone Shop,Grocery Store,Latin American Restaurant,Sandwich Place,Wine Shop,Spanish Restaurant,New American Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Café,Mexican Restaurant,Pizza Place,Lounge,Wine Bar,Park,Bakery,Frozen Yogurt Shop,Spanish Restaurant,Deli / Bodega
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Yoga Studio,Indian Restaurant,Park,Sushi Restaurant,Sandwich Place,Deli / Bodega


In [47]:
map_clusters = folium.Map(location = [lati, longi], zoom_start = 11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for la, lo, ne, lab in zip(Final_hoods['Latitude'], Final_hoods['Longitude'], Final_hoods['Neighborhood'], Final_hoods['Labels']):
  label = folium.Popup(str(ne) + 'Cluster' + str(lab), parse_html = True)
  folium.CircleMarker([la, lo], radius = 5, popup = label, color = rainbow[lab - 1], fill = True, fill_color = rainbow[lab-1], fill_opacity = 0.7).add_to(map_clusters)

map_clusters


In [54]:
#Cluster 1
Final_hoods.loc[Final_hoods['Labels'] == 0, Final_hoods.columns[[1] + list(range(5, Final_hoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2rd Most Common Venue,3nd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,Stuyvesant Town,Boat or Ferry,Park,Bar,Playground,Bistro,Basketball Court,Baseball Field,Harbor / Marina,Farmers Market,Cocktail Bar


In [55]:
#Cluster 2
Final_hoods.loc[Final_hoods['Labels'] == 1, Final_hoods.columns[[1] + list(range(5, Final_hoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2rd Most Common Venue,3nd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Sandwich Place,Coffee Shop,Yoga Studio,Discount Store,Miscellaneous Shop,Steakhouse,Supplement Shop,Seafood Restaurant,Tennis Stadium,Gym
1,Chinatown,Chinese Restaurant,Cocktail Bar,American Restaurant,Vietnamese Restaurant,Bakery,Bubble Tea Shop,Spa,Optical Shop,Salon / Barbershop,Ice Cream Shop
2,Washington Heights,Café,Bakery,Deli / Bodega,Mobile Phone Shop,Grocery Store,Latin American Restaurant,Sandwich Place,Wine Shop,Spanish Restaurant,New American Restaurant
3,Inwood,Café,Mexican Restaurant,Pizza Place,Lounge,Wine Bar,Park,Bakery,Frozen Yogurt Shop,Spanish Restaurant,Deli / Bodega
4,Hamilton Heights,Mexican Restaurant,Café,Pizza Place,Coffee Shop,Yoga Studio,Indian Restaurant,Park,Sushi Restaurant,Sandwich Place,Deli / Bodega
5,Manhattanville,Coffee Shop,Italian Restaurant,Deli / Bodega,Liquor Store,Mexican Restaurant,Seafood Restaurant,Park,Sushi Restaurant,Supermarket,Spanish Restaurant
6,Central Harlem,African Restaurant,Chinese Restaurant,Bar,American Restaurant,French Restaurant,Seafood Restaurant,Gym / Fitness Center,Public Art,Boutique,Beer Bar
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Pizza Place,Latin American Restaurant,Thai Restaurant,Sandwich Place,Coffee Shop,Cocktail Bar,Donut Shop
9,Yorkville,Italian Restaurant,Gym,Bar,Coffee Shop,Pizza Place,Deli / Bodega,Sushi Restaurant,Diner,Japanese Restaurant,Wine Shop
11,Roosevelt Island,Park,Sandwich Place,Coffee Shop,Gym,Bus Line,Supermarket,Outdoors & Recreation,Farmers Market,Greek Restaurant,Scenic Lookout


In [56]:
#Cluster 3
Final_hoods.loc[Final_hoods['Labels'] == 2, Final_hoods.columns[[1] + list(range(5, Final_hoods.shape[1]))]]

,Neighborhood,1st Most Common Venue,2rd Most Common Venue,3nd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Italian Restaurant,Exhibit,Art Gallery,Bakery,Coffee Shop,Juice Bar,French Restaurant,Gym / Fitness Center,Hotel,Spa
10,Lenox Hill,Italian Restaurant,Coffee Shop,Pizza Place,Sushi Restaurant,Cosmetics Shop,Burger Joint,Café,Gym,Gym / Fitness Center,Sporting Goods Shop
12,Upper West Side,Italian Restaurant,Wine Bar,Coffee Shop,Indian Restaurant,Bakery,Bar,Vegetarian / Vegan Restaurant,Sushi Restaurant,Pet Store,Middle Eastern Restaurant
13,Lincoln Square,Theater,Gym / Fitness Center,Café,Plaza,Concert Hall,Italian Restaurant,French Restaurant,Park,Performing Arts Venue,Indie Movie Theater
14,Clinton,Theater,Gym / Fitness Center,Spa,American Restaurant,Italian Restaurant,Wine Shop,Hotel,Sandwich Place,Coffee Shop,Bar
15,Midtown,Hotel,Coffee Shop,Cocktail Bar,Clothing Store,Theater,Steakhouse,Bakery,Bookstore,Sporting Goods Shop,Japanese Restaurant
16,Murray Hill,Coffee Shop,Hotel,Japanese Restaurant,Sandwich Place,Italian Restaurant,French Restaurant,Gym / Fitness Center,American Restaurant,Gym,Bakery
17,Chelsea,Coffee Shop,Italian Restaurant,Bakery,Ice Cream Shop,Nightclub,Hotel,Theater,American Restaurant,Seafood Restaurant,Cycle Studio
18,Greenwich Village,Italian Restaurant,Clothing Store,Sushi Restaurant,Café,Seafood Restaurant,Indian Restaurant,Cosmetics Shop,Chinese Restaurant,French Restaurant,Vietnamese Restaurant
21,Tribeca,Italian Restaurant,American Restaurant,Park,Boutique,Spa,Café,Greek Restaurant,Coffee Shop,Gym,Wine Bar
